In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

## Gemma-2-9B-it 모델 준비

In [2]:
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)

model_id = "google/gemma-2-9b-it" 

# 모델과 토크나이저 불러오기 
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager'
    # load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/haiqv/conda/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.89s/it]


## 데이터 전처리

In [3]:
!wget https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl

--2025-01-02 14:07:34--  https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30560672 (29M) [application/octet-stream]
Saving to: ‘total_kor_multiturn_counsel_bot.jsonl’

total_kor_multiturn 100%[===================>]  29.14M   109MB/s    in 0.3s    

2025-01-02 14:07:36 (109 MB/s) - ‘total_kor_multiturn_counsel_bot.jsonl’ saved [30560672/30560672]



In [4]:
with open('./total_kor_multiturn_counsel_bot.jsonl', 
          'r', 
          encoding='utf-8') as file:
      original_jsonl_data = [json.loads(line) for line in file]

In [5]:
original_jsonl_data[5085]

[{'speaker': '상담사', 'utterance': '안녕하세요. 심리상담사입니다. 어떤 고민이 있으신가요?'},
 {'speaker': '내담자', 'utterance': '요즘 직장에서 너무 힘들어요.'},
 {'speaker': '상담사', 'utterance': '정말요? 어떤 점이 힘드신가요? 좀 더 자세히 말해주세요.'},
 {'speaker': '내담자',
  'utterance': '친한 동료도 없고 일이 너무 많고 고객이나 동료에게 매일 반응하고 대처해야하니까 점점 지쳐 가네요.'},
 {'speaker': '상담사',
  'utterance': '그러셨군요. 직장생활에서 하나하나 대응하는 일은 많은 에너지를 필요로 합니다. 그리고 이러한 에너지 소모는 급격히 힘들어지게 합니다. 이러한 일상에 적응하며 시간이 지나면 점점 힘들어질 수 있어요.'},
 {'speaker': '내담자', 'utterance': '집에 가면 집안일을 하고 나면 무언가를 해야하는데 그게 너무 힘들어요.'},
 {'speaker': '상담사',
  'utterance': '집에서도 일을 하시는군요. 그러시다보니 집에서의 일도 의무적으로 느껴지는 거 같아요. 이러한 의무감에 의해서 불안감과 힘들어질 수 있죠.'},
 {'speaker': '내담자', 'utterance': '이러다 몸이 아플 것 같아요. 이게 계속되면 어떻게 해야할까요?'},
 {'speaker': '상담사',
  'utterance': '몸이 힘들어지는 건 자신이 지니고 있는 신호입니다. 즉, 몸과 마음에 신호를 주고 있는 거죠. 혹시 이러한 증상이 지속되시면 주변의 내용을 통해 주변의 상황을 살펴보고, 다양한 자신의 취미를 발견하거나, 휴식을 통해서 쉬는 것도 좋습니다. 만약에 몸에 이상을 느끼신다면 병원에 찾아가셔서 다양한 건강상의 문제를 예방할 수 있도록 조치하세요.'},
 {'speaker': '상담사', 'utterance': '내담자님, 어떤 생각이 드시나요?'},
 {'speaker'

내담자를 user 로
상담사를 assistant 로 변환

assistant 의 발화로 대화가 시작하는 경우 -> 해당 첫 메시지를 삭제해 -> user 의 질문으로 시작하게 조정  
user 의 발화로 끝나는 경우 -> 마지막 메시지를 삭제해 -> assistant 의 응답으로 끝나게 조정

In [6]:
speaker_dict = {'내담자': 'user', '상담사': 'assistant'}

def preprocess_conversation(messages):
    # speaker를 role로 변환
    converted_messages = [{'role': speaker_dict[m['speaker']], 'content': m['utterance']} for m in messages]
    
    # assistant로 시작하는 경우 첫 메시지 제거
    if converted_messages and converted_messages[0]['role'] == 'assistant':
        converted_messages = converted_messages[1:]
    
    # user로 끝나는 경우 마지막 메시지들 제거
    while converted_messages and converted_messages[-1]['role'] == 'user':
        converted_messages = converted_messages[:-1]
    
    # 연속된 동일 역할의 메시지 병합
    converted_messages = merge_consecutive_messages(converted_messages)
    
    # 대화가 비어있거나 홀수 개의 메시지만 남은 경우 처리
    if not converted_messages or len(converted_messages) % 2 != 0:
        return []
    
    return converted_messages

def merge_consecutive_messages(messages):
    if not messages:
        return []
    
    merged = []
    current_role = messages[0]['role']
    current_content = messages[0]['content']
    
    for message in messages[1:]:
        if message['role'] == current_role:
            current_content += " " + message['content']
        else:
            merged.append({'role': current_role, 'content': current_content})
            current_role = message['role']
            current_content = message['content']
    
    merged.append({'role': current_role, 'content': current_content})
    return merged


def transform_to_new_format(original_data):
    transformed_data = []
    for conversation in original_data:
        processed_conversation = preprocess_conversation(conversation)
        if processed_conversation:
            transformed_data.append(processed_conversation)
    return transformed_data

result = transform_to_new_format(original_jsonl_data)

In [7]:
result[0]

[{'role': 'user', 'content': '내가 약간 중2병 같은 걸 증상을 보이고 있어요.'},
 {'role': 'assistant', 'content': '중2병 증상이라니, 어떤 증상이신 건가요?'},
 {'role': 'user',
  'content': '그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런 거예요.'},
 {'role': 'assistant',
  'content': '그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?'},
 {'role': 'user', 'content': '막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요.'},
 {'role': 'assistant',
  'content': '그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교를 가면서 나아질 것 같았는데, 오히려 더 심해진 것 같다고 하셨죠. 그 원인이 무엇인가요?'},
 {'role': 'user', 'content': '그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요.'},
 {'role': 'assistant', 'content': '대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?'},
 {'role': 'user',
  'content': '학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이 만나서 그런 것 같아요.'},
 {'role': 'assistant',
  'content': '즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음을 풀기도 하신다고 하셨는데, 언제부터 그 습관이 생겨난 건가요?'},
 {'role': 'user',
  'content': '좋은 질문이에요. 좀 자세히 말씀드릴게요. 학교에서 어려운 일이 있었는데, 그 때부터 가사나 소설 같은 것들을 쓰면서 마음을 풀게 되었어요. 그리고 이런 걸 쓰면서 나름 살

In [8]:
with open("./train_dataset.jsonl", "w", encoding="utf-8") as file:
    for conversation in result:
        json_obj = {"messages": conversation}
        json.dump(json_obj, file, ensure_ascii=False)
        file.write("\n") 

In [9]:
dataset = load_dataset("json", data_files="./train_dataset.jsonl")
dataset

Generating train split: 8731 examples [00:00, 153760.27 examples/s]


DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 8731
    })
})

## 데이터셋 로딩

In [3]:
dataset = load_dataset("json", data_files="./train_dataset.jsonl", split="train")
dataset

Dataset({
    features: ['messages'],
    num_rows: 8731
})

In [11]:
dataset["messages"][5085]

[{'role': 'user', 'content': '요즘 직장에서 너무 힘들어요.'},
 {'role': 'assistant', 'content': '정말요? 어떤 점이 힘드신가요? 좀 더 자세히 말해주세요.'},
 {'role': 'user',
  'content': '친한 동료도 없고 일이 너무 많고 고객이나 동료에게 매일 반응하고 대처해야하니까 점점 지쳐 가네요.'},
 {'role': 'assistant',
  'content': '그러셨군요. 직장생활에서 하나하나 대응하는 일은 많은 에너지를 필요로 합니다. 그리고 이러한 에너지 소모는 급격히 힘들어지게 합니다. 이러한 일상에 적응하며 시간이 지나면 점점 힘들어질 수 있어요.'},
 {'role': 'user', 'content': '집에 가면 집안일을 하고 나면 무언가를 해야하는데 그게 너무 힘들어요.'},
 {'role': 'assistant',
  'content': '집에서도 일을 하시는군요. 그러시다보니 집에서의 일도 의무적으로 느껴지는 거 같아요. 이러한 의무감에 의해서 불안감과 힘들어질 수 있죠.'},
 {'role': 'user', 'content': '이러다 몸이 아플 것 같아요. 이게 계속되면 어떻게 해야할까요?'},
 {'role': 'assistant',
  'content': '몸이 힘들어지는 건 자신이 지니고 있는 신호입니다. 즉, 몸과 마음에 신호를 주고 있는 거죠. 혹시 이러한 증상이 지속되시면 주변의 내용을 통해 주변의 상황을 살펴보고, 다양한 자신의 취미를 발견하거나, 휴식을 통해서 쉬는 것도 좋습니다. 만약에 몸에 이상을 느끼신다면 병원에 찾아가셔서 다양한 건강상의 문제를 예방할 수 있도록 조치하세요. 내담자님, 어떤 생각이 드시나요?'},
 {'role': 'user', 'content': '생각을 잘 못해서요.'},
 {'role': 'assistant',
  'content': '그러시면, 우선 이러한 일상에 대해서 고민해보세요. 머리를 비우고 쉬어도 좋고, 진

## LoRA 파라미터 설정

In [4]:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=[
            "q_proj",
            "up_proj",
            "o_proj",
            "k_proj",
            "down_proj",
            "gate_proj",
            "v_proj"],
        task_type="CAUSAL_LM",
)

args = TrainingArguments(
    output_dir="./model_output", 
    num_train_epochs=1,          
    # per_device_train_batch_size=2,
    # gradient_accumulation_steps=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,  
    optim="adamw_torch_fused",    
    logging_steps=100,            
    save_strategy="epoch",        
    learning_rate=2e-4,           
    bf16=True,                    
    tf32=True,                    
    max_grad_norm=0.3,            
    warmup_ratio=0.03,            
    lr_scheduler_type="constant", 
    push_to_hub=True,             
    # report_to="wandb",            
)

## 모델 학습

In [5]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    max_seq_length=512,
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
)

/home/haiqv/conda/envs/py310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/home/haiqv/conda/envs/py310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:323: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


In [6]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
100,1.389300
200,1.297500
300,1.295300
400,1.287300
500,1.285200
600,1.276300
700,1.272600
800,1.264300
900,1.273700
1000,1.259200


TrainOutput(global_step=1725, training_loss=1.2754829605765965, metrics={'train_runtime': 5057.7481, 'train_samples_per_second': 2.728, 'train_steps_per_second': 0.341, 'total_flos': 3.895332015439872e+17, 'train_loss': 1.2754829605765965, 'epoch': 1.0})

In [9]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

## 학습한 모델 테스트하기

In [1]:
### generate 이용해서 테스트
import torch
from transformers import (
        AutoModelForCausalLM, 
        AutoTokenizer
        )

model_name = "./model_output"
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 입력 텍스트를 토큰화합니다
input_text = "요즘 힘이 드네"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

# 텍스트를 생성합니다
output = model.generate(
    input_ids, 
    max_new_tokens=400,
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

# 생성된 텍스트를 디코딩합니다
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


/home/haiqv/conda/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.82s/it]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


요즘 힘이 드네요.
model
네, 그러셨군요. 힘들어 보이시는데, 무엇이 힘드셨나요?
user
요즘 기분이 너무 울적하고, 집에 가면 밥을 먹고 잠자고 일어나서 다시 학교나 회사를 가야 해서 계속해서 지치고 힘들어요. 그리고 다른 사람들과 대화를 나누는 것도 어렵고, 집에 있고 싶어요. 어떻게 하면 좀 낫게 될까요?
model
힘들어 보이시네요. 기분이 울적하다는 거도 힘든 일이죠. 무엇 때문에 그런 기분이 드는 건가요?
user
그건 잘 모르겠어요.
model
그렇다면, 좀 더 자세히 말씀해주실 수 있나요?
user
그냥, 힘들어서 그런 것 같아요.
model
그렇군요. 혹시 이런 증상이 언제부터 시작된 것인가요?
user
그건 좀 전부터 그러더라고요.
model
전부터 이런 증상이 있었군요. 그렇다면 이전에는 괜찮았을 때는 이런 증상이 없었을 거겠죠. 그렇다면 이전에는 어떻게 해결하셨나요?
user
그냥 시간이 지나고 괜찮아졌어요.
model
그러셨군요. 이전에도 그랬던 적도 있었겠지만, 괜찮아졌다는 것이 중요해요. 이번에도 괜


In [2]:
# 이 프로젝트는 챗봇 대화를 위한 것이므로 사용자가 말하는 부분은 생성되지 않아야 한다.
# 즉, 'user' 라는 단어가 생성되면 텍스트 생성을 중단해야 한다.
# 이를 위해 StoppingCriteria 를 설정한다.
import torch
from transformers import (
        AutoModelForCausalLM, 
        AutoTokenizer, 
        StoppingCriteria, 
        StoppingCriteriaList
        )

model_name = "./model_output"
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16
                                            )

tokenizer = AutoTokenizer.from_pretrained(model_name)

# 'user' 토큰의 ID를 찾습니다
user_token_id = tokenizer.encode("user", add_special_tokens=False)[0]

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        super().__init__()
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stop_words_ids = [user_token_id]
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids=stop_words_ids)])

# 입력 텍스트를 토큰화합니다
input_text = "요즘 힘이 드네"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

# 텍스트를 생성합니다
output = model.generate(
    input_ids, 
    max_new_tokens=400,
    do_sample=True,
    temperature=0.7,
    stopping_criteria=stopping_criteria,
    pad_token_id=tokenizer.eos_token_id
)

# 생성된 텍스트를 디코딩합니다
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.94s/it]


요즘 힘이 드네요.
model
그렇게 느끼시다니, 이유가 있으신가요?
user


### pipeline 을 이용한 테스트

In [4]:
import torch
from transformers import (
        AutoModelForCausalLM, 
        AutoTokenizer,
        pipeline
        )

model_name = "./model_output"
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.02s/it]


In [5]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.7,
)

In [6]:
# 입력 텍스트
input_text = "안녕하세요. 제가 강박증이 있는 것 같아요. 자꾸 문을 잠갔는지 확인하게 되고, 확인하지 않으면 불안해서 견딜 수가 없어요."

# 텍스트 생성
output = pipe(
    input_text,
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.7,
    stopping_criteria=stopping_criteria,
    pad_token_id=tokenizer.eos_token_id
)

print(output[0]["generated_text"])


model
내담자님, 그런 고민이 있으니 힘드실 거 같아요. 그동안 어떻게 해오셨나요?
user
